# MyAcrobot
my custom acrobot rl model for learning, this will have an extended custom environment

### Create my virutal environment

#

pip install virtualenv
virtualenv myenv
source myenv/bin/activate
pip install numpy pandas matplotlib
pip install ipykernel
python -m ipykernel install --user --name=myenv --display-name "Python (myenv)"



pip install virtualenv virtualenv myenv source myenv/bin/activate pip install numpy pandas matplotlib pip install ipykernel python -m ipykernel install --user --name=myenv --display-name "Python (myenv)" 